In [ ]:
# mudanças: correção na extração dos dados

!pip install pdfplumber pdf2image pytesseract pillow
!apt-get install -y poppler-utils tesseract-ocr tesseract-ocr-por

import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import re
import pandas as pd
from google.colab import files

# DEBUG = True imprime trechos para inspeção
DEBUG = False

# Upload dos arquivos PDF
uploaded = files.upload()

def extrair_texto(pdf_path):
    """Tenta extrair texto com pdfplumber; se uma página vier vazia, usa OCR."""
    texto_total = ""
    with pdfplumber.open(pdf_path) as pdf:
        for pagina in pdf.pages:
            txt = pagina.extract_text()
            if txt and txt.strip():
                texto_total += txt + "\n"
            else:
                # fallback para OCR na página
                img = pagina.to_image(resolution=200).original
                try:
                    texto_total += pytesseract.image_to_string(img, lang="por") + "\n"
                except:
                    texto_total += pytesseract.image_to_string(img, lang="eng") + "\n"
    return texto_total

LOCAIS_ACEITOS = [
    "Centro de Tecnologia",
    "Centro de Ciencias Juridicas e Economicas",
    "Centro de Ciencias Matematicas e da Natureza",
    "Centro de Ciencias da Saúde",
    "Campus Duque de Caxias",
    "Centro de Filosofia e Ciencias Humanas",
    "Centro de Letras e Artes",
    "Multidisciplinar UFRJ-Macaé",
    "Forum de Ciencia e Cultura",
    "Campus Macaé",
    "Polo Xerem"
]

def normalizar_local(texto_local):
    """
    Dado um texto extraído do PDF, tenta encontrar algum local válido da lista.
    Retorna o nome padronizado, se achar; caso contrário, devolve o texto cru.
    """
    if not texto_local:
        return "N/D"

    texto_local_norm = texto_local.lower()

    for local in LOCAIS_ACEITOS:
        if local.lower() in texto_local_norm:
            return local  # devolve o nome padronizado (exato da lista)

    return texto_local.strip()

def normalizar_texto(texto):
    """Normalizações seguras:
       - compressa espaços
       - garante espaços ao redor de palavras-chave (ARTIGO, TITULO, PARTICIPANTES, etc.)
       - corrige casos onde o número vem antes do token (ex: '1596ARTIGO' -> 'ARTIGO 1596')
    """
    if not texto:
        return texto

    # 1) substituir qualquer whitespace (newline/tabs) por um espaço simples
    texto = re.sub(r'\s+', ' ', texto)

    # 2) lista de keywords que queremos separar e padronizar (sem acentos para simplificar)
    keywords = [
        "SESSÕES", "SESSOES", "ARTIGO", "TRABALHO", "TITULO", "TÍTULO",
        "PARTICIPANTES", "AUTORES", "ORIENTADORES", "EQUIPE",
        "ORAL", "PÔSTER", "POSTER", "LOCAL"
    ]

    # Insere espaços ao redor das keywords (case-insensitive)
    for kw in keywords:
        # usa regex com flags para não quebrar; adiciona espaços antes/depois
        texto = re.sub(fr'(?i){re.escape(kw)}', f' {kw} ', texto)

    # 3) Se houver padrão "1234 ARTIGO" ou "1234ARTIGO" (número antes), inverter para "ARTIGO 1234"
    texto = re.sub(r'(\d{1,6})\s*(?=(?:ARTIGO|TRABALHO)\b)', r'\g<0>', texto)  # mantém se já com espaço
    # agora troca "NUM ARTIGO" -> "ARTIGO NUM"
    texto = re.sub(r'(\d{1,6})\s+(ARTIGO|TRABALHO)\b', r'\2 \1', texto, flags=re.IGNORECASE)
    # também trata caso totalmente colado "1596ARTIGO" -> "ARTIGO 1596"
    texto = re.sub(r'(\d{1,6})(?=(ARTIGO|TRABALHO)\b)', r' \2 \1', texto, flags=re.IGNORECASE)

    # 4) novamente compressa espaços múltiplos para 1
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

def extrair_trabalhos(pdf_path, nome_pdf):
    trabalhos = []
    texto_total = extrair_texto(pdf_path)
    texto_total = normalizar_texto(texto_total)

    if DEBUG:
        print("=== Trecho inicial do texto normalizado ===")
        print(texto_total[:1200])
        print("=== fim trecho ===\n")

    # Local (procura "Sessões - ...")
    m = re.search(r'\bSESS(?:ÕES|OES)\b\s*-\s*(.*?)\b(?:ARTIGO|TRABALHO|ORAL|PÔSTER|POSTER|LOCAL)\b', texto_total, flags=re.IGNORECASE)
    if m:
        local_arquivo = m.group(1).strip()
    else:
        # fallback: procura primeiro "LOCAL:" no texto
        m2 = re.search(r'\bLOCAL\s*[:\-]?\s*(.*?)(?:ARTIGO|TRABALHO|ORAL|PÔSTER|POSTER)\b', texto_total, flags=re.IGNORECASE)
        local_arquivo = m2.group(1).strip() if m2 else "N/D"

    local_arquivo = normalizar_local(local_arquivo)

    # Regex robusta para capturar blocos; aceita ARTIGO ou TRABALHO, TITULO/TÍTULO,
    # aceita : - ou nada, e para autores usa várias variantes.
    padrao = re.findall(
        r"(?:ARTIGO|TRABALHO)\s*[:\-]?\s*(\d{1,6})\s+T[IÍ]TULO\s*[:\-]?\s*(.*?)\s+"
        r"(?:AUTORES(?:\s+E\s+ORIENTADORES)?|PARTICIPANTES|ORIENTADORES|EQUIPE)\s*[:\-]?\s*"
        r"(.*?)(?=(?:ARTIGO|TRABALHO)\b|(?:APRESENTA[Çç][aã]O\b)|(?:(?:MINICURSO|OFICINA)\b)|(?:ORAL\b)|(?:P[oô]STER\b)|\bLOCAL\b|$)", #analisa se ta dando problema com o numero '|\d+\b'
        texto_total,
        flags=re.DOTALL | re.IGNORECASE | re.UNICODE
    )

    # ver quantos matches foram achados (DEBUG)
    if DEBUG:
        print("Matches encontrados:", len(padrao))

    for num, titulo, autores in padrao:
        titulo_clean = re.sub(r'\s+', ' ', titulo).strip()
        titulo_clean = re.sub(r'</?p>', '', titulo_clean, flags=re.IGNORECASE).strip()
        autores_clean = re.sub(r'\s+', ' ', autores).strip()
        # remove números isolados (sem colar em letras)
        autores_clean = re.sub(r'\b\d+\b', '', autores_clean).strip()
        suspeita = False
        if autores_clean == "" or len(autores_clean) < 3:
            suspeita = True

        trabalhos.append({
            "Local": local_arquivo,
            "Trabalho": num.strip(),
            "Título": titulo_clean,
            "Autores/Participantes": autores_clean,
        })

    return trabalhos

# Processar PDFs enviados
todos_trabalhos = []
for nome_arquivo in uploaded.keys():
    print(f"Processando: {nome_arquivo}")
    todos_trabalhos.extend(extrair_trabalhos(nome_arquivo, nome_arquivo))

# Consolidar CSV
df = pd.DataFrame(todos_trabalhos)
csv_name = "siac_.csv"
df.to_csv(csv_name, index=False, encoding="utf-8-sig")
files.download(csv_name)

print(f"Total de trabalhos extraídos: {len(df)}")
#if 'Suspeita' in df.columns:
#    print(f"Registros suspeitos: {df['Suspeita'].sum()}")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-por is already the newest version (1:4.00~git30-7274cfa-1.1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.10).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


Saving siac_ccje_2017.pdf to siac_ccje_2017.pdf
Saving siac_ccmn_2017.pdf to siac_ccmn_2017.pdf
Saving siac_ccs_2017.pdf to siac_ccs_2017.pdf
Saving siac_cfch_2017.pdf to siac_cfch_2017.pdf
Saving siac_cla_2017.pdf to siac_cla_2017.pdf
Saving siac_cmm_2017.pdf to siac_cmm_2017.pdf
Saving siac_ct_2017.pdf to siac_ct_2017.pdf
Saving siac_fcc_2017.pdf to siac_fcc_2017.pdf
Saving siac_px_2017.pdf to siac_px_2017.pdf
Processando: siac_ccje_2017.pdf
Processando: siac_ccmn_2017.pdf
Processando: siac_ccs_2017.pdf
Processando: siac_cfch_2017.pdf
Processando: siac_cla_2017.pdf
Processando: siac_cmm_2017.pdf
Processando: siac_ct_2017.pdf
Processando: siac_fcc_2017.pdf
Processando: siac_px_2017.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Total de trabalhos extraídos: 4814


In [ ]:
# versao anterior do codigo :/
!pip install pdfplumber pdf2image pytesseract pillow
!apt-get install -y poppler-utils tesseract-ocr tesseract-ocr-por

import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import re
import pandas as pd
from google.colab import files

# DEBUG = True imprime trechos para inspeção
DEBUG = False

# Upload dos arquivos PDF
uploaded = files.upload()

def extrair_texto(pdf_path):
    """Tenta extrair texto com pdfplumber; se uma página vier vazia, usa OCR."""
    texto_total = ""
    with pdfplumber.open(pdf_path) as pdf:
        for pagina in pdf.pages:
            txt = pagina.extract_text()
            if txt and txt.strip():
                texto_total += txt + "\n"
            else:
                # fallback para OCR na página
                img = pagina.to_image(resolution=200).original
                try:
                    texto_total += pytesseract.image_to_string(img, lang="por") + "\n"
                except:
                    texto_total += pytesseract.image_to_string(img, lang="eng") + "\n"
    return texto_total

def normalizar_texto(texto):
    """Normalizações seguras:
       - compressa espaços
       - garante espaços ao redor de palavras-chave (ARTIGO, TITULO, PARTICIPANTES, etc.)
       - corrige casos onde o número vem antes do token (ex: '1596ARTIGO' -> 'ARTIGO 1596')
    """
    if not texto:
        return texto

    # 1) substituir qualquer whitespace (newline/tabs) por um espaço simples
    texto = re.sub(r'\s+', ' ', texto)

    # 2) lista de keywords que queremos separar e padronizar (sem acentos para simplificar)
    keywords = [
        "SESSÕES", "SESSOES", "ARTIGO", "TRABALHO", "TITULO", "TÍTULO",
        "PARTICIPANTES", "AUTORES", "ORIENTADORES", "EQUIPE",
        "ORAL", "PÔSTER", "POSTER", "LOCAL"
    ]

    # Insere espaços ao redor das keywords (case-insensitive)
    for kw in keywords:
        # usa regex com flags para não quebrar; adiciona espaços antes/depois
        texto = re.sub(fr'(?i){re.escape(kw)}', f' {kw} ', texto)

    # 3) Se houver padrão "1234 ARTIGO" ou "1234ARTIGO" (número antes), inverter para "ARTIGO 1234"
    texto = re.sub(r'(\d{1,6})\s*(?=(?:ARTIGO|TRABALHO)\b)', r'\g<0>', texto)  # mantém se já com espaço
    # agora troca "NUM ARTIGO" -> "ARTIGO NUM"
    texto = re.sub(r'(\d{1,6})\s+(ARTIGO|TRABALHO)\b', r'\2 \1', texto, flags=re.IGNORECASE)
    # também trata caso totalmente colado "1596ARTIGO" -> "ARTIGO 1596"
    texto = re.sub(r'(\d{1,6})(?=(ARTIGO|TRABALHO)\b)', r' \2 \1', texto, flags=re.IGNORECASE)

    # 4) novamente compressa espaços múltiplos para 1
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

def extrair_trabalhos(pdf_path, nome_pdf):
    trabalhos = []
    texto_total = extrair_texto(pdf_path)
    texto_total = normalizar_texto(texto_total)

    if DEBUG:
        print("=== Trecho inicial do texto normalizado ===")
        print(texto_total[:1200])
        print("=== fim trecho ===\n")

    # Local (procura "Sessões - ...")
    m = re.search(r'\bSESS(?:ÕES|OES)\b\s*-\s*(.*?)\b(?:ARTIGO|TRABALHO|ORAL|PÔSTER|POSTER|LOCAL)\b', texto_total, flags=re.IGNORECASE)
    if m:
        local_arquivo = m.group(1).strip()
    else:
        # fallback: procura primeiro "LOCAL:" no texto
        m2 = re.search(r'\bLOCAL\s*[:\-]?\s*(.*?)(?:ARTIGO|TRABALHO|ORAL|PÔSTER|POSTER)\b', texto_total, flags=re.IGNORECASE)
        local_arquivo = m2.group(1).strip() if m2 else "N/D"

    # Regex robusta para capturar blocos; aceita ARTIGO ou TRABALHO, TITULO/TÍTULO,
    # aceita : - ou nada, e para autores usa várias variantes.
    padrao = re.findall(
        r"(?:ARTIGO|TRABALHO)\s*[:\-]?\s*(\d{1,6})\s+T[IÍ]TULO\s*[:\-]?\s*(.*?)\s+"
        r"(?:AUTORES(?:\s+E\s+ORIENTADORES)?|PARTICIPANTES|ORIENTADORES|EQUIPE)\s*[:\-]?\s*"
        r"(.*?)(?=(?:ARTIGO|TRABALHO)\b|(?:APRESENTA[Çç][aã]O\b)|(?:(MINICURSO|OFICINA)\b)|(?:ORAL\b)|(?:P[oô]STER\b)|\bLOCAL\b|\d+\b|$)",
        texto_total,
        flags=re.DOTALL | re.IGNORECASE | re.UNICODE
    )

    # ver quantos matches foram achados (DEBUG)
    if DEBUG:
        print("Matches encontrados:", len(padrao))

    for num, titulo, autores in padrao:
        titulo_clean = re.sub(r'\s+', ' ', titulo).strip()
        autores_clean = re.sub(r'\s+', ' ', autores).strip()
        suspeita = False
        if autores_clean == "" or len(autores_clean) < 3:
            suspeita = True

        trabalhos.append({
            "Local": local_arquivo,
            "Trabalho": num.strip(),
            "Título": titulo_clean,
            "Autores/Participantes": autores_clean,
        })

    return trabalhos

# Processar PDFs enviados
todos_trabalhos = []
for nome_arquivo in uploaded.keys():
    print(f"Processando: {nome_arquivo}")
    todos_trabalhos.extend(extrair_trabalhos(nome_arquivo, nome_arquivo))

# Consolidar CSV
df = pd.DataFrame(todos_trabalhos)
csv_name = "siac_.csv"
df.to_csv(csv_name, index=False, encoding="utf-8-sig")
files.download(csv_name)

print(f"Total de trabalhos extraídos: {len(df)}")
#if 'Suspeita' in df.columns:
#    print(f"Registros suspeitos: {df['Suspeita'].sum()}")
